In [1]:
import nltk
from nltk.corpus import brown
#from nltk.tokenize import word_tokenize
import numpy as np
#import pandas as pd
import re
import copy
#import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
import random #for shuffling list
from sklearn.model_selection import train_test_split
import tensorflow as tf
import gensim

nltk.download('punkt')
nltk.download('brown')
nltk.download('universal_tagset')
dataset = brown.tagged_sents(tagset='universal')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


In [2]:
import gensim.downloader as api
import json
info = api.info()
#print(json.dumps(info, indent=4))
print(api.load('word2vec-google-news-300', return_path=True))

[==================================================] 100.0% 1662.8/1662.8MB downloaded
/root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz


In [3]:
w2v_model = gensim.models.KeyedVectors.load_word2vec_format('/root/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz', binary=True)

In [4]:
import os
import torch
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

In [5]:
words=[]
tags=[]
for sent_id in range(len(dataset)):
    words_list=[]
    tags_list=[]
    for i in range(len(dataset[sent_id])):
        words_list.append(dataset[sent_id][i][0])
        tags_list.append(dataset[sent_id][i][1])
    words.append(words_list)
    tags.append(tags_list)

train_words,test_words, train_tags, test_tags=train_test_split(words,tags,test_size=0.2,train_size=0.8)

words_in_corpus = []
tags_in_corpus = []

for words_list in train_words:
    for item in words_list:
        words_in_corpus.append(item)
for tags_list in train_tags:
    for item in tags_list:
        tags_in_corpus.append(item)

words_in_corpus = list(set(words_in_corpus))
tags_in_corpus = list(set(tags_in_corpus))

words_in_corpus.sort()
tags_in_corpus.sort()

print(len(words_in_corpus))
print(tags_in_corpus)

word_dictionary = dict()
tag_dictionary = dict()

for i in range ( len(tags_in_corpus) ):
    tag_dictionary[tags_in_corpus[i]] = i # 0 based indexing of IDs

for i in range ( len(words_in_corpus) ):
    word_dictionary[words_in_corpus[i]] = i # 0 based indexing of IDs

index_for_word_not_in_corpus = len(words_in_corpus)

50646
['.', 'ADJ', 'ADP', 'ADV', 'CONJ', 'DET', 'NOUN', 'NUM', 'PRON', 'PRT', 'VERB', 'X']


In [6]:
from torch.nn.utils.rnn import pad_sequence
class dataset_pos_ED(Dataset):

    def __init__(self,words,tags,device):
        
        self.words=words
        self.tags=tags
        self.device=device

    def __len__(self):
        return len(self.words)
    

    def feature_matrix(self,words):
        f_mat=[]
        for i in words:
            try:
                f_mat.append(torch.Tensor(w2v_model[i]).unsqueeze(dim=0))

            except:

                x=np.random.normal(loc=0.0,scale=0.1,size=300)
                f_mat.append(torch.Tensor(x).unsqueeze(dim=0))
        f_mat=torch.cat(f_mat,axis=0).to(self.device)
        return f_mat

    def __getitem__(self, idx):
        # sentence=[]
        # tag_list=[]
        words=self.words[idx]
        tags=self.tags[idx]
        # tagged_sent=self.data[idx]
        # for item in tagged_sent:
            # sentence.append(item[0])
            # tag_list.append(item[1])
        feature=self.feature_matrix(words)
        return (feature), torch.Tensor([int(tag_dictionary[i]) for i in tags])

In [7]:
data_train=dataset_pos_ED(train_words,train_tags,"cpu")
def pad_collate(batch):
    (xx, yy) = zip(*batch)
    x_lens = [len(x) for x in xx]
    y_lens = [len(y) for y in yy]

    xx_pad = pad_sequence(xx, batch_first=True, padding_value=11)
    yy_pad = pad_sequence(yy, batch_first=True, padding_value=11)

    return xx_pad, yy_pad
dataloader = DataLoader(data_train, batch_size=100,shuffle=True, num_workers=0, collate_fn=pad_collate)


In [8]:
import torch.nn as nn
class encoder(nn.Module):
    def __init__(self,hidden_dim=150,layer_dim=1,in_dim=300,out_dim=12,device="cuda"):
        super().__init__()
        self.hidden_dim=hidden_dim
        self.layer_dim=layer_dim
        self.in_dim=in_dim
        self.out_dim=out_dim
        self.device=device
        self.rnn1 = nn.RNN(in_dim, hidden_dim, layer_dim, batch_first=True, nonlinearity='relu',bidirectional=True)        
        # self.softmax=nn.Softmax(dim=-1).to(device)
        # self.lin=nn.Linear(2*hidden_dim,out_dim)

    def forward(self,x):
        out,hn=self.rnn1(x)
        # x=self.lin(out)
        # x=self.softmax(x)
        return torch.cat((hn[-1],hn[-2]),axis=1)

In [9]:
import torch.nn as nn
class decoder(nn.Module):
    def __init__(self,hidden_dim=300,layer_dim=1,in_dim=12,out_dim=12,device="cuda"):
        super().__init__()
        self.hidden_dim=hidden_dim
        self.layer_dim=layer_dim
        self.in_dim=in_dim
        self.out_dim=out_dim
        self.device=device
        self.rnn1 = nn.RNN(in_dim, hidden_dim, layer_dim, batch_first=True, nonlinearity='relu',bidirectional=True)        
        self.softmax=nn.Softmax(dim=-1).to(device)
        self.lin=nn.Linear(2*hidden_dim,out_dim)

    def forward(self,x,seq_length):
        predictions=[]
        
        for i in range(seq_length):
            if i==0:

                input=torch.zeros(x.shape[0],1,12)
                input[:,:,0]=torch.ones(x.shape[0],1)
                
                # print("input shape",input.shape)
                x=x.unsqueeze(dim=0).expand(2*self.layer_dim,x.shape[0],x.shape[1])
                # print("x shape", x.shape)
                out,hn=self.rnn1(input,x)
                x=hn
            else:
                input=torch.zeros(x.shape[0],1,12)
                input=torch.nn.functional.one_hot(input_item.long(),num_classes=12).to(device).unsqueeze(1).float()
                # input[input_item]=1
                # print("next iter input shape",input.shape)
                # print("hn shape",hn.shape)
                out,hn=self.rnn1(input,hn)
            a=self.softmax(self.lin(out.squeeze()))
            if i==0:
                b=a.unsqueeze(1)
            else:
                b=torch.cat((b,a.unsqueeze(1)),axis=1 )
            # print("a shape",a.shape)
            predictions.append(a.argmax(1))
            input_item=predictions[-1]
            # print("input_item shape",input_item.shape)


        
        # out,hn=self.rnn1(x)
        # x=self.lin(out)
        # x=self.softmax(x)
        return predictions, b

In [10]:
class Model_ED(nn.Module):
    def __init__(self,device="cuda"):
        super().__init__()
        self.encoder=encoder(device=device)
        self.decoder=decoder(device=device)
    def forward(self,x,seq_length):
        x=self.encoder(x)
        # print("enocder shape", x.shape)
        # print(seq_length)
        preds,outs=self.decoder(x,seq_length)
        return preds,outs

In [13]:
import time
device="cuda"
# dataloader = DataLoader(data_train, batch_size=100,
                        # shuffle=True, num_workers=0)
model=Model_ED(device=device)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
loss_fn=nn.CrossEntropyLoss()
model.train()
prev_end=time.time()
num_epochs=10
for e in range(num_epochs):
    total_loss=0.0
    for batch, (X, y) in enumerate(dataloader):
        size = len(dataloader.dataset)
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        # print(X.shape)
        # print(y.shape)
        preds,outs = model(X,X.shape[1])
        # print(pred.shape)
        # print(y)
        y=torch.nn.functional.one_hot(y.long(),num_classes=12).to(device).float()
        # print(pred.shape)
        # print(y.shape)
        loss = loss_fn(outs, y)
        total_loss+=loss
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 10 == 0:
            end=time.time()
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            print("time: ", end-prev_end)
            rev_end=end
    print("Total loss for epoch {}".format(e+1), total_loss)
            

RuntimeError: ignored

In [ ]:
def train_func(data_train, data_test):
    import time
    device="cuda"
    dataloader = DataLoader(data_train, batch_size=100,
                            shuffle=True, num_workers=0)
    model=encoder(device=device)
    optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)
    loss_fn=nn.CrossEntropyLoss()
    model.train()
    start=time.time()
    num_epochs=1
    for e in range(num_epochs):

        for batch, (X, y) in enumerate(dataloader):
            size = len(dataloader.dataset)
            X, y = X.to(device), y.to(device)

            # Compute prediction error
            pred = model(X)
            # print(pred.shape)
            # print(y.shape)
            loss = loss_fn(pred, y)

            # Backpropagation
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            break
            if batch % 500 == 0:
                end=time.time()
                loss, current = loss.item(), (batch + 1) * len(X)
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
                print("time: ", end-start)
            break
        break
    
    

    dataloader = DataLoader(data_test, batch_size=100,
                            shuffle=True, num_workers=0)
       
    a = torch.tensor([]).to(device)
    b = torch.tensor([]).to(device)

    for batch, (X, y) in enumerate(dataloader):
        size = len(dataloader.dataset)
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        # print(pred.shape)
        # print(y.shape)
        loss = loss_fn(pred, y)
        a = torch.cat((a, y))
        b = torch.cat((b, torch.argmax(pred, axis = 1)))

    return a, b

In [ ]:
def accuracy_per_fold(train_w, train_t, test_w, test_t):

    train_words,train_tags=train_w, train_t
    # train_words,test_words, train_tags, test_tags=train_test_split(words,tags,test_size=0.2,train_size=0.8)

    words_in_corpus = []
    tags_in_corpus = []

    for words_list in train_words:
        for item in words_list:
            words_in_corpus.append(item)
    for tags_list in train_tags:
        for item in tags_list:
            tags_in_corpus.append(item)

    words_in_corpus = list(set(words_in_corpus))
    tags_in_corpus = list(set(tags_in_corpus))

    words_in_corpus.sort()
    tags_in_corpus.sort()

    print(len(words_in_corpus))
    print(tags_in_corpus)

    word_dictionary = dict()
    tag_dictionary = dict()

    for i in range ( len(tags_in_corpus) ):
        tag_dictionary[tags_in_corpus[i]] = i # 0 based indexing of IDs

    for i in range ( len(words_in_corpus) ):
        word_dictionary[words_in_corpus[i]] = i # 0 based indexing of IDs

    index_for_word_not_in_corpus = len(words_in_corpus)



    data_train = dataset_pos_ED(train_w,train_t,"cuda")
    data_test = dataset_pos_ED(test_w,test_t,"cuda")
    test_y,predictions = train_func(data_train, data_test)
    
    f1 = 0
    prec = 0
    recall = 0
    

    #for i in range( len(actual_tags) ):
    #    f1 += f1_score(actual_tags[i], predicted_tags[i], average='weighted')
    #    prec += precision_score(actual_tags[i], predicted_tags[i], average='weighted')
    #    recall += recall_score(actual_tags[i], predicted_tags[i], average='weighted')
    per_pos=[[] for i in range(12)]
    #per_pos_act=[[] for i in range(0,12)]
    for i in range(len(test_y)):
        per_pos[test_y[i].to(torch.int64).item()].append(predictions[i].to(torch.int64).item())
    print("my_fold------------------------")
    for i in range(len(per_pos)):
        print(i,"-----")
        true_tag=[i for j in range(len(per_pos[i]))]
        print("precision",precision_score(per_pos[i],true_tag, average='weighted'))
        print("recall",recall_score(per_pos[i],true_tag, average='weighted'))
        print("f1",f1_score(per_pos[i],true_tag, average='weighted'))
    print("overall for the batch:")
    precision=precision_score(test_y.to('cpu'),predictions.to('cpu'), average='weighted')
    recall=recall_score(test_y.to('cpu'),predictions.to('cpu'), average='weighted')
    f1=f1_score(test_y.to('cpu'),predictions.to('cpu'), average='weighted')
    print("precision",precision)
    print("recall",recall)
    print("f1",f1)

        
    return [precision, recall, f1]
    #return [f1/len(actual_tags), prec/len(actual_tags), recall/len(actual_tags)]

#total_acc_for_split1 = accuracy_per_fold(all_sentences[:57000], all_sentences[56000:56010])
#total_acc_for_split2 = accuracy_per_fold(all_sentences[:57000], all_sentences[57000:57010])

#print("\nTotal accuracy for the above split1: " + str(total_acc_for_split1))
#print("Total accuracy for the above split2: " + str(total_acc_for_split2) + "\n")
#exit(0)

def cross_validation_accuracy(words, tags):
    all_accuracies = []
    total_acc = [0, 0, 0]

    num_sentences = len(words)

    fold_size = int(num_sentences / 5)

    print(fold_size)

    fold1_train_w=words[:4*fold_size]
    fold1_test_w=words[4*fold_size:]
    fold2_train_w=words[:3*fold_size]+words[4*fold_size:]
    fold2_test_w=words[3*fold_size:4*fold_size]
    fold3_train_w=words[:2*fold_size]+words[3*fold_size:]
    fold3_test_w=words[2*fold_size:3*fold_size]
    fold4_train_w=words[:1*fold_size]+words[2*fold_size:]
    fold4_test_w=words[1*fold_size:2*fold_size]
    fold5_train_w=words[1*fold_size:]
    fold5_test_w=words[:1*fold_size]

    fold1_train_t=tags[:4*fold_size]
    fold1_test_t=tags[4*fold_size:]
    fold2_train_t=tags[:3*fold_size]+tags[4*fold_size:]
    fold2_test_t=tags[3*fold_size:4*fold_size]
    fold3_train_t=tags[:2*fold_size]+tags[3*fold_size:]
    fold3_test_t=tags[2*fold_size:3*fold_size]
    fold4_train_t=tags[:1*fold_size]+tags[2*fold_size:]
    fold4_test_t=tags[1*fold_size:2*fold_size]
    fold5_train_t=tags[1*fold_size:]
    fold5_test_t=tags[:1*fold_size]

    all_accuracies.append(accuracy_per_fold(fold1_train_w,fold1_train_t, fold1_test_w, fold1_test_t))
    print("Accuracy for fold ", "1", " is : ", str(all_accuracies[0]))
    total_acc[0] += all_accuracies[0][0]
    total_acc[1] += all_accuracies[0][1]
    total_acc[2] += all_accuracies[0][2]


    all_accuracies.append(accuracy_per_fold(fold2_train_w,fold2_train_t, fold2_test_w, fold2_test_t))
    print("Accuracy for fold ", "2", " is : ", str(all_accuracies[1]))
    total_acc[0] += all_accuracies[1][0]
    total_acc[1] += all_accuracies[1][1]
    total_acc[2] += all_accuracies[1][2]

    all_accuracies.append(accuracy_per_fold(fold3_train_w,fold3_train_t, fold3_test_w, fold3_test_t))
    print("Accuracy for fold ", "3", " is : ", str(all_accuracies[2]))
    total_acc[0] += all_accuracies[2][0]
    total_acc[1] += all_accuracies[2][1]
    total_acc[2] += all_accuracies[2][2]

    all_accuracies.append(accuracy_per_fold(fold4_train_w,fold4_train_t, fold4_test_w, fold4_test_t))
    print("Accuracy for fold ", "4", " is : ", str(all_accuracies[3]))
    total_acc[0] += all_accuracies[3][0]
    total_acc[1] += all_accuracies[3][1]
    total_acc[2] += all_accuracies[3][2]

    all_accuracies.append(accuracy_per_fold(fold5_train_w,fold5_train_t, fold5_test_w, fold5_test_t))
    print("Accuracy for fold ", "5", " is : ", str(all_accuracies[4]))
    total_acc[0] += all_accuracies[4][0]
    total_acc[1] += all_accuracies[4][1]
    total_acc[2] += all_accuracies[4][2]

    print("Cross validation accuracy is : ", str(np.array(total_acc) / 5))

In [ ]:
cross_validation_accuracy(train_words, train_tags)